In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import datetime

In [2]:
data = pd.read_csv('ks-projects-201801.csv')
data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [3]:
data.describe()

,ID,goal,pledged,backers,usd_pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [26]:
# The keyspace should have already been created using `cqlsh`:
# ---
# cqlsh> CREATE KEYSPACE kickstarter WITH replication = {'class':'SimpleStrategy', 'replication_factor':1};
# ---
# Also, the `projects` table should have been created already (with correct data types):
# ```
# cqlsh> CREATE TABLE IF NOT EXISTS kickstarter.projects (
#                ... ID bigint PRIMARY KEY,
#                ... name varchar,
#                ... category varchar,
#                ... main_category varchar,
#                ... currency varchar,
#                ... deadline date,
#                ... goal decimal,
#                ... launched timestamp,
#                ... pledged decimal,
#                ... state varchar,
#                ... backers int,
#                ... country varchar,
#                ... usd_pledged decimal,
#                ... usd_pledged_real decimal,
#                ... usd_goal_real decimal);
# ```
cluster = Cluster(port=9042)
session = cluster.connect('kickstarter')

In [27]:
column_names = list(data.columns.values)
names_str = column_names[0]
for name in column_names[1:]:
    names_str += ',' + name
names_str

'ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real'

In [33]:
# Create a prepared statement to insert rows into Cassandra.
insert_str = 'INSERT INTO projects (' + names_str + ') VALUES (?'
for _ in range(1, len(column_names)):
    insert_str += ',?'
insert_str += ')'
insert_stmt = session.prepare(insert_str)
items = data.to_dict('list')
# Convert columns to the correct data types.
items['name'] = [v if v is str else '' for v in items['name']]
items['launched'] = [datetime.datetime.strptime(items['launched'][i], '%Y-%m-%d %H:%M:%S').date() for i in range(len(items['launched']))]
items['usd_pledged'] = [str(v).encode('utf-8') if v is float else '0.00' for v in items['usd_pledged']]
# Add all of the items to the table.
projects = []
for i in range(len(items[column_names[0]])):
    item = [items[column_names[j]][i] for j in range(len(column_names))]
    project = session.execute(insert_stmt, item)
    projects.append(project)
len(projects)

378661

In [35]:
result = session.execute('SELECT COUNT(*) FROM projects;')
result.current_rows

[Row(count=378661)]